In this version we are doing bayesian optimization instead of deterministic optimization

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import plotly
import torch_geometric.data as data
from torch_geometric.loader import DataLoader
from tqdm import tqdm

from networkit import *
import networkx as nx
import nxviz as nv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.metrics import f1_score

import torch
from torch.nn import Linear
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch.nn.functional import normalize
from torch_geometric.datasets import WebKB, StochasticBlockModelDataset
from torch_geometric.utils import negative_sampling, to_networkx, to_dense_adj
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, GraphConv, TransformerConv   
from torch_geometric.utils import train_test_split_edges
import matplotlib.pyplot as plt

/home/siddy/anaconda3/envs/GDL/lib/python3.8/site-packages/nxviz/__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.4 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.4, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = StochasticBlockModelDataset(root="/home/siddy/META/data", block_sizes=[10,10], edge_probs=[[0.3, 0.1], [0.1, 0.3]], is_undirected=True)
data = dataset[0].to(device)

In [4]:
data

Data(edge_index=[2, 66], y=[20])

In [5]:
def neg_index(data):
    num_nodes = data.num_nodes
    edge_index = data.edge_index

    # Convert edge indices to adjacency matrix
    adj_matrix = torch.zeros((num_nodes, num_nodes), dtype=torch.long)
    for i in range(edge_index.size(1)):
        u, v = int(edge_index[0, i]), int(edge_index[1, i])
        adj_matrix[u, v] = 1
        adj_matrix[v, u] = 1

    neg_indices = []
    for i in range(edge_index.size(1)):
        u, v = int(edge_index[0, i]), int(edge_index[1, i])
        adj_squared = torch.mm(adj_matrix, adj_matrix)
        adj_row_u = adj_matrix[u]
        adj_row_v = adj_matrix[v]
        bitwise_and_result = torch.bitwise_and(adj_squared[u], adj_squared[v])
        sum_bitwise_and_result = torch.sum(bitwise_and_result)
        sum_adj_u = torch.sum(adj_row_u)
        sum_adj_v = torch.sum(adj_row_v)
        neg_index = sum_bitwise_and_result.float() / (sum_adj_u + sum_adj_v)
        neg_indices.append(neg_index.item())

    return neg_indices

In [6]:
def pos_index(data):
    num_nodes = data.num_nodes
    edge_index = data.edge_index

    adj_matrix = torch.zeros((num_nodes, num_nodes), dtype=torch.long)
    for i in range(edge_index.size(1)):
        u, v = int(edge_index[0, i]), int(edge_index[1, i])
        adj_matrix[u, v] = 1
        adj_matrix[v, u] = 1

    pos_indices = []
    for i in range(edge_index.size(1)):
        u, v = int(edge_index[0, i]), int(edge_index[1, i])
        adj_row_u = adj_matrix[u]
        adj_row_v = adj_matrix[v]
        bitwise_and_result = torch.bitwise_and(adj_row_u, adj_row_v)
        sum_bitwise_and_result = torch.sum(bitwise_and_result)
        sum_adj_u = torch.sum(adj_row_u)
        sum_adj_v = torch.sum(adj_row_v)
        pos_index = sum_bitwise_and_result.float() / (sum_adj_u + sum_adj_v)
        pos_indices.append(pos_index.item())

    return pos_indices

In [12]:
#torch.clamp(sigma_1 - 0.5, min=0) + sigma_2
def custom_objective(sigma_1, sigma_2):
    return (sigma_1 - 0.5)**2 + sigma_2

def create_variational_matrix_individual(data, num_nodes, edge_index):
    variational_matrix = torch.nn.Parameter(torch.rand((num_nodes, num_nodes), dtype=torch.float), requires_grad=True)
    optimizer = optim.Adam([variational_matrix], lr=0.001, weight_decay=5e-4)

    
    for epoch in range(1000):
        optimizer.zero_grad()

        # Calculate sigma_1 and sigma_2 for each e
        neg_indices = neg_index(data)
        pos_indices = pos_index(data)

        total_loss = 0
        for i in range(edge_index.size(1)):
            u, v = int(edge_index[0, i]), int(edge_index[1, i])
            # current pair
            sigma_1 = torch.tensor(neg_indices[i], requires_grad=True)
            sigma_2 = torch.tensor(pos_indices[i], requires_grad=True)
            objective = custom_objective(sigma_1, sigma_2)
            loss = torch.min(objective)
            # Accumulate loss for all pairs
            total_loss += loss

        total_loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch + 1}, Total Loss: {total_loss.item()}")

    # Sampling from the learned variational matrix to generate an adjacency matrix
    adjacency_matrix = torch.sigmoid(variational_matrix)  # Sigmoid to constrain values between 0 and 1

    # Thresholding to convert probabilities into binary adjacency matrix
    #adjacency_matrix_binary = (adjacency_matrix > 0.5).float()
    #edge_index = torch.nonzero(adjacency_matrix, as_tuple=True)

    return adjacency_matrix


Trying out BOtorch

In [7]:
import plotly.figure_factory as ff
neg_indices = neg_index(data)
pos_indices = pos_index(data)

hist_data = [neg_indices, pos_indices]
group_labels=['neg_indices', 'pos_indices']
colors = ['#A56CC1', '#B8F7D4']
fig = ff.create_distplot(hist_data, group_labels, colors=colors, bin_size=.2,
                         show_curve=True)
fig.update_layout(title_text='Disitribition of Neg vs Pos indices')
fig.show()

/home/siddy/anaconda3/envs/GDL/lib/python3.8/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [8]:
#let see the function we want to optimize
def custom_objective(sigma_1, sigma_2):
    return (sigma_1 - 0.5)**2 + sigma_2

losses= []
for i in range(data.edge_index.size(1)):
    sigma_1 = torch.tensor(neg_indices[i], requires_grad=True)
    sigma_2 = torch.tensor(pos_indices[i], requires_grad=True)
    objective = custom_objective(sigma_1, sigma_2)
    loss = torch.min(objective)
    losses.append(loss.item())

variational_matrix = torch.nn.Parameter(torch.rand((20, 20), dtype=torch.float), requires_grad=True)

In [9]:
import plotly.graph_objects as go
data = go.Scatter(x=losses, y=variational_matrix.detach().numpy(), line_color='red')

fig = go.Figure(data=data)
fig.update_layout(title="initial variational matrix vs losses", xaxis_title="losses", yaxis_title="variational matrix")
fig.show()

In [22]:
torch.tensor(losses).shape

torch.Size([66])

In [23]:
# botorch input type
train_x = variational_matrix.to(torch.float64)
exact_obj = torch.tensor(losses)
best_observed_value = min(losses)

In [17]:
bounds = torch.tensor([[0.], [1.]])

In [25]:
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.models.transforms.outcome import Standardize


In [26]:
gp = SingleTaskGP(train_x, exact_obj, outcome_transform=Standardize(m=1))
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)



RuntimeError: Wrong output dimension. Y.size(-1) is 66; expected 1.